In [1]:
pip install ptflops

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from ptflops import get_model_complexity_info

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 50, 5)
        self.fc1 = nn.Linear(50 * 4 * 4, 800)
        self.fc2 = nn.Linear(800, 500)
        self.fc3 = nn.Linear(500, 10)
        self.a_type = 'relu'
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        layer1 = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        layer2 = F.max_pool2d(F.relu(self.conv2(layer1)), 2)
        layer2_p = layer2.view(-1, int(layer2.nelement() / layer2.shape[0]))
        layer3 = F.relu(self.fc1(layer2_p))
        layer4 = F.relu(self.fc2(layer3))
        layer5 = self.fc3(layer4)
        return layer5

In [6]:
# Initialize your model
model = LeNet()

# Load the checkpoint file
checkpoint_path = "base.pth"
checkpoint = th.load(checkpoint_path)

# Inspect the contents of the checkpoint
print("Checkpoint keys:", checkpoint.keys())

# If the checkpoint is a state_dict itself (most common case)
if isinstance(checkpoint, dict):
    print("\nVariables in the checkpoint:")
    for key, value in checkpoint.items():
        print(f"{key}: {type(value)}")

# Load the model state dictionary
model.load_state_dict(checkpoint['model'])

Checkpoint keys: dict_keys(['model', 'train_acc', 'test_acc', 'optimizer', 'scheduler'])

Variables in the checkpoint:
model: <class 'collections.OrderedDict'>
train_acc: <class 'float'>
test_acc: <class 'float'>
optimizer: <class 'dict'>
scheduler: <class 'dict'>


<All keys matched successfully>

In [7]:
# Download and transform train dataset
trainloader = th.utils.data.DataLoader(datasets.MNIST('../data',
                                                      download=True,
                                                      train=True,
                                                      transform=transforms.Compose([
                                                          transforms.ToTensor(),
                                                      ])),
                                       batch_size=100,
                                       shuffle=True, num_workers=0)

# Download and transform test dataset
testloader = th.utils.data.DataLoader(datasets.MNIST('../data',
                                                     download=True,
                                                     train=False,
                                                     transform=transforms.Compose([
                                                         transforms.ToTensor(),
                                                     ])),
                                      batch_size=100,
                                      shuffle=True, num_workers=0)

In [20]:
# Compute FLOPs for only convolutional layers
ignore_modules = [nn.Linear, nn.Softmax]

In [21]:
# Compute FLOPs and parameters
with th.cuda.device(0):
    macs, params = get_model_complexity_info(model, (1, 28, 28), as_strings=True,
                                             print_per_layer_stat=True, verbose=True, 
                                             ignore_modules=ignore_modules, backend='pytorch')
    print(f"Computational complexity (Conv layers only): {macs}")
    print(f"Number of parameters (Conv layers only): {params}")

LeNet(
  25.57 k, 2.386% Params, 1.9 MMac, 98.410% MACs, 
  (conv1): Conv2d(520, 0.049% Params, 299.52 KMac, 15.491% MACs, 1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(25.05 k, 2.337% Params, 1.6 MMac, 82.919% MACs, 20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(0, 0.000% Params, 0.0 Mac, 0.000% MACs, in_features=800, out_features=800, bias=True)
  (fc2): Linear(0, 0.000% Params, 0.0 Mac, 0.000% MACs, in_features=800, out_features=500, bias=True)
  (fc3): Linear(0, 0.000% Params, 0.0 Mac, 0.000% MACs, in_features=500, out_features=10, bias=True)
  (softmax): Softmax(0, 0.000% Params, 0.0 Mac, 0.000% MACs, dim=1)
)
Computational complexity (Conv layers only): 1.93 MMac
Number of parameters (Conv layers only): 1.07 M


In [16]:
# Evaluate the model
model.eval()
correct = 0
total = 0

with th.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = th.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 99.02%


TypeError: unsupported operand type(s) for /: 'str' and 'float'